In [ ]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
audio_original  = '/content/drive/MyDrive/Dataset/Parkinson_Positive'

In [ ]:
snr_db=20

In [ ]:
for file in os.listdir(audio_original):
    audio_path = os.path.join(audio_original,file)
    audio,sr=sf.read(audio_path)
    noise_samples=np.random.randn(len(audio))
    signal_power= np.mean(audio**2)
    noise_power = np.mean(noise_samples**2)
    scale_factor= np.sqrt(signal_power/(noise_power*(10**(snr_db/10))))
    noisy_audio = audio + scale_factor*noise_samples
    augmented_filename= f"augmented_{file}"
    audio_generated = '/content/drive/MyDrive/Dataset/audio_generated_positive'
    augmented_filepath = os.path.join(audio_generated,augmented_filename)
    sf.write(augmented_filepath,noisy_audio,sr)

In [ ]:
import librosa
time_stretch_factor=1.5
for file in os.listdir(audio_original):
    file_path = os.path.join(audio_original, file)
    audio,sr = librosa.load(file_path,sr=4000)
    time_stretched= librosa.effects.time_stretch(audio, rate=time_stretch_factor)
    output_file = f"time_stretched_{file}"
    audio_generated = '/content/drive/MyDrive/Dataset/audio_generated_positive'
    output_path = os.path.join(audio_generated, output_file)
    sf.write(output_path, time_stretched, sr)

In [ ]:

import os
import shutil

def copy_files(source_folder, destination_folder):
    # Get a list of all audio files in the source folder
    audio_files = [file for file in os.listdir(source_folder) if file.endswith('.wav')]

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for audio_file in audio_files:
        # Get the source and destination file paths
        source_path = os.path.join(source_folder, audio_file)
        destination_path = os.path.join(destination_folder, audio_file)

        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)

def main():
    # Set the paths to the source and destination folders
    source_folder = r'C:\Users\AL56170\Downloads\Parkinson_dataset\Parkinson_negative'
    destination_folder = 'u'

    # Call the copy_files function
    copy_files(source_folder, destination_folder)

if __name__ == '__main__':
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\AL56170\\Downloads\\Parkinson_dataset\\Parkinson_negative'

In [ ]:
audio_original = '/content/drive/MyDrive/Dataset/audio_generated_positive'
target_duration = 200
for file in os.listdir(audio_original):
    if file.endswith(".wav"):  # Adjust file extensions as needed
        file_path = os.path.join(audio_original, file)
        audio,sr = librosa.load(file_path,sr=None)
        current_duration=librosa.get_duration(y=audio,sr=sr)
        if current_duration<target_duration:
            pad_length = int((target_duration - current_duration)*sr)
            padded_audio=np.pad(audio,(0,pad_length), mode='constant')
            sf.write(file_path,padded_audio,sr)
        elif current_duration>target_duration:
            num_samples = int(target_duration*sr)
            truncated_audio = audio[:num_samples]
            sf.write(file_path, truncated_audio, sr)

In [ ]:
audio_folder_positive = '/content/drive/MyDrive/Dataset/audio_generated_positive'
audio_folder_negative = '/content/drive/MyDrive/Dataset/audio_generated_negative'
n_mfcc = 15
hop_length=512
win_length=512
positive_features = []
negative_features = []
for file_name in os.listdir(audio_folder_positive):
    if file_name.endswith('.wav'):
        audio_path = os.path.join(audio_folder_positive, file_name)
        audio, sr = librosa.load(audio_path, sr=4000)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, win_length=win_length)
        positive_features.append(mfcc.T)  # Transpose for shape compatibility

for file_name in os.listdir(audio_folder_negative):
    if file_name.endswith('.wav'):
        audio_path = os.path.join(audio_folder_negative, file_name)
        audio, sr = librosa.load(audio_path, sr=4000)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, win_length=win_length)
        negative_features.append(mfcc.T)  # Transpose for shape compatibility
positive_features = np.array(positive_features)
negative_features = np.array(negative_features)
print("Combined Features Shape:", positive_features.shape)
print("Combined Features Shape:", negative_features.shape)

Combined Features Shape: (100, 1563, 15)
Combined Features Shape: (100, 1563, 15)


In [ ]:
X_p = positive_features.reshape(positive_features.shape[0],-1)
X_n = negative_features.reshape(negative_features.shape[0],-1)
X__ = np.concatenate((X_p,X_n),axis=0)

In [ ]:
X__.shape

(200, 23445)

In [ ]:
Y=np.concatenate((np.ones(len(positive_features)),np.zeros(len(negative_features))),axis=0)

In [ ]:
Y.shape

(200,)

In [ ]:
audio_folder_1 = '/content/drive/MyDrive/Dataset/audio_generated_positive'
audio_folder_2 = '/content/drive/MyDrive/Dataset/audio_generated_negative'
spectral_centroid_features_folder1 = []
spectral_bandwidth_features_folder1 = []
spectral_contrast_features_folder1 = []

spectral_centroid_features_folder2 = []
spectral_bandwidth_features_folder2 = []
spectral_contrast_features_folder2 = []

def extract_spectral_features(audio_folder):
    spectral_centroid_features = []
    spectral_bandwidth_features = []
    spectral_contrast_features = []
    for file_name in os.listdir(audio_folder):
        if file_name.endswith('.wav'):
            audio_path = os.path.join(audio_folder, file_name)
            audio, sr = librosa.load(audio_path, sr=4000)
            spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
            spectral_centroid_features.append(spectral_centroid)
            spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
            spectral_bandwidth_features.append(spectral_bandwidth)
    spectral_centroid_features = np.array(spectral_centroid_features)
    spectral_bandwidth_features = np.array(spectral_bandwidth_features)
    spectral_centroid_features= np.squeeze(spectral_centroid_features)
    spectral_bandwidth_features= np.squeeze(spectral_bandwidth_features)
    return spectral_centroid_features, spectral_bandwidth_features,
spectral_centroid_features_folder1, spectral_bandwidth_features_folder1 = extract_spectral_features(audio_folder_1)
spectral_centroid_features_folder2, spectral_bandwidth_features_folder2 = extract_spectral_features(audio_folder_2)
print("Spectral Centroid Features Folder 1 Shape:", spectral_centroid_features_folder1.shape)
print("Spectral Bandwidth Features Folder 1 Shape:", spectral_bandwidth_features_folder1.shape)


print("Spectral Centroid Features Folder 2 Shape:", spectral_centroid_features_folder2.shape)
print("Spectral Bandwidth Features Folder 2 Shape:", spectral_bandwidth_features_folder2.shape)




Spectral Centroid Features Folder 1 Shape: (100, 1563)
Spectral Bandwidth Features Folder 1 Shape: (100, 1563)
Spectral Centroid Features Folder 2 Shape: (100, 1563)
Spectral Bandwidth Features Folder 2 Shape: (100, 1563)


In [ ]:
X_p = np.concatenate((spectral_centroid_features_folder1,spectral_bandwidth_features_folder1),axis=1)
X_n = np.concatenate((spectral_centroid_features_folder2,spectral_bandwidth_features_folder2),axis=1)
X__ = np.concatenate((X_p,X_n),axis=0)
X__.shape
X__

array([[903.65969806, 934.22455826, 947.29523103, ...,   0.        ,
          0.        ,   0.        ],
       [940.29156512, 936.44226579, 947.64601305, ...,   0.        ,
          0.        ,   0.        ],
       [943.6792746 , 935.86956728, 958.54356358, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [683.9226433 , 704.28011654, 811.34543725, ...,   0.        ,
          0.        ,   0.        ],
       [564.46929751, 537.77630344, 490.17336415, ...,   0.        ,
          0.        ,   0.        ],
       [528.60326985, 445.02499798, 451.35623032, ...,   0.        ,
          0.        ,   0.        ]])

In [ ]:
scaler = StandardScaler()
scaler.fit(X__)
X__input = scaler.transform(X__)
X__input

array([[ 1.04986607,  1.22982936,  1.36467668, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [ 1.21730794,  1.2403189 ,  1.36636994, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [ 1.23279294,  1.23761009,  1.41897334, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       ...,
       [ 0.04546235,  0.14221437,  0.70843545, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [-0.50055109, -0.64533257, -0.84188952, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [-0.66449236, -1.08403728, -1.02926313, ..., -0.10048225,
        -0.10047323, -0.1004206 ]])

In [ ]:
import joblib

In [ ]:
scaler = StandardScaler()
scaler.fit(X__)
X_input = scaler.transform(X__)
joblib.dump(scaler,"scaler.pkl")

['scaler.pkl']

In [ ]:
X_input = np.concatenate((X_input,X__input),axis=1)
X_input

array([[ 1.04986607,  1.22982936,  1.36467668, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [ 1.21730794,  1.2403189 ,  1.36636994, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [ 1.23279294,  1.23761009,  1.41897334, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       ...,
       [ 0.04546235,  0.14221437,  0.70843545, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [-0.50055109, -0.64533257, -0.84188952, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [-0.66449236, -1.08403728, -1.02926313, ..., -0.10048225,
        -0.10047323, -0.1004206 ]])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,cross_val_predict

X_train,X_test, Y_train,Y_test=train_test_split(X_input,Y,test_size=0.2,random_state=40)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
classifier = SVC(kernel = 'linear',random_state = 42)
classifier.fit(X_train,Y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(Y_test,y_pred)
accuracy

0.75

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(Y_test,y_pred)

In [ ]:
accuracy

0.75

In [ ]:
final_dataset = np.concatenate((X_input,Y.reshape(-1,1)),axis=1)
final_dataset

array([[ 1.04986607,  1.22982936,  1.36467668, ..., -0.10047323,
        -0.1004206 ,  1.        ],
       [ 1.21730794,  1.2403189 ,  1.36636994, ..., -0.10047323,
        -0.1004206 ,  1.        ],
       [ 1.23279294,  1.23761009,  1.41897334, ..., -0.10047323,
        -0.1004206 ,  1.        ],
       ...,
       [ 0.04546235,  0.14221437,  0.70843545, ..., -0.10047323,
        -0.1004206 ,  0.        ],
       [-0.50055109, -0.64533257, -0.84188952, ..., -0.10047323,
        -0.1004206 ,  0.        ],
       [-0.66449236, -1.08403728, -1.02926313, ..., -0.10047323,
        -0.1004206 ,  0.        ]])

In [ ]:
np.random.shuffle(final_dataset)

In [ ]:
X_final = final_dataset[:,:-1]
X_final

array([[ 0.23893247,  0.62108464,  1.03661235, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [-1.40134681, -1.26906509, -1.13418672, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [-1.6654982 , -1.81745344, -1.89306927, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       ...,
       [ 1.08072382,  1.20263737,  1.42357812, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [ 0.13548329, -0.04967412, -0.07899414, ..., -0.10048225,
        -0.10047323, -0.1004206 ],
       [-0.21778734,  0.11640413,  0.26129464, ..., -0.10048225,
        -0.10047323, -0.1004206 ]])

In [ ]:
Y_final = final_dataset[:,-1]
Y_final.shape

(200,)

In [ ]:
#performing cross validation
model=SVC()
model_predict=cross_val_predict(model,X_final,Y_final,cv=10)
model_score=cross_val_score(model,X_final,Y_final,cv=10)

In [ ]:
model_score.mean()


0.7799999999999999

In [ ]:
model_predict

array([0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1.])

In [ ]:
from sklearn.metrics import confusion_matrix,f1_score

confusion_result = confusion_matrix(Y_final,model_predict)
confusion_result

array([[73, 27],
       [17, 83]])

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
Y_final

array([0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.])

In [ ]:
X_final.shape

(200, 6252)

In [ ]:
Y_final.shape


(200,)

In [ ]:
from sklearn.metrics import classification_report
classification_result  = classification_report(Y_test,y_pred)
classification_result

'              precision    recall  f1-score   support\n\n         0.0       0.87      0.62      0.72        21\n         1.0       0.68      0.89      0.77        19\n\n    accuracy                           0.75        40\n   macro avg       0.77      0.76      0.75        40\nweighted avg       0.78      0.75      0.75        40\n'

In [ ]:
confusion_result = confusion_matrix(Y_test,y_pred)
confusion_result

array([[13,  8],
       [ 2, 17]])